In [1]:
import cv2 
import os 
import glob
import json
import numpy as np
import scipy
import skimage
import matplotlib.pyplot as plt 
import pandas as pd

from numpy import matlib
from scipy.ndimage import convolve
from skimage.measure import label, regionprops_table
from tqdm import tqdm 
from module.utils import imread
from module.disp_measure import convert_by_img
from itertools import product

import time




In [2]:
sensor_num = 59
sensor_num = str(sensor_num)


img_list = glob.glob(os.path.join(r'imgs', sensor_num, '*.jpg'))

with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)
    


In [ ]:
params = param_config[sensor_num]

result = [] 
src_img = imread(img_list[0])

for img_path in tqdm(img_list): 
    dest_img = imread(img_path)
    disp, dest_circles = convert_by_img(dest_img,
                                 src_img,
                                 min_rad = params['min_rad'],
                                 max_rad = params['max_rad'],
                                 param1 = params['param1'],
                                 param2 = params['param2'])
    result.append(disp)
    dest_img = cv2.cvtColor(dest_img, cv2.COLOR_BGR2GRAY)
    _, dest_img = cv2.threshold(dest_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    for dest_circle in dest_circles : 
        dest_img = cv2.circle(dest_img, tuple(dest_circle[:2]), int(dest_circle[2]), (0,255,0), thickness=1, lineType=8, shift=0)
#     dest_img = cv2.cvtColor(dest_img, cv2.COLOR_BGR2GRAY)
#     dest_img = cv2.adaptiveThreshold(dest_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,10)

    cv2.imwrite(img_path[:-4] + '_with_circles_result.jpeg', dest_img)
    
collect_rslt = []
for rslt in result : 
    collect_rslt.append(rslt[0])
    
plt.plot(np.array(collect_rslt))

 31%|████████████████████████▋                                                       | 260/843 [02:00<04:30,  2.15it/s]

In [45]:
## check the circle detection accuracy 

param1 = 200
param2 = 14
min_rad = 77
max_rad = 83
img = imread(img_list[0]) 

# ret3, grey_img = cv2.threshold(grey_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grey_img = cv2.adaptiveThreshold(grey_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY,11,10)


In [45]:
# th3 = 255 - th3

circles = cv2.HoughCircles(grey_img, 
                           cv2.HOUGH_GRADIENT,
                           1, 
                           max_rad*2, 
                           param1=param1,
                           param2=param2,
                           minRadius=min_rad,
                           maxRadius=max_rad)

# Draw circles that are detected. 
if circles is not None: 
  
    # Convert the circle parameters a, b and r to integers. 
    circles = np.uint16(np.around(circles)) 
  
    for pt in circles[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        cv2.circle(img, (a, b), r, (0, 255, 0), 2) 
  
        # Draw a small circle (of radius 1) to show the center. 
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3) 
        cv2.imshow("Detected Circle", img) 
        cv2.waitKey(0) 
else:
        cv2.imshow('Circle is not detected', img)
        cv2.waitKey(0) 

NameError: name 'grey_img' is not defined

In [3]:
def getGrayImage(img) : 

    N = img.ndim
    if N == 3 : # If RGB Image, 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img.dtype == np.uint8:    
            img = np.asarray(img, dtype = np.float)/255
    elif N == 2: 
        if img.dtype == np.uint8:    
            img = np.asarray(img, dtype = np.float)/255
    return img 



In [4]:
def imgradient(img): 

    hy = -np.asarray([[1.0, 2.0, 1.0], [0, 0, 0], [-1.0, -2.0, -1.0]])
    hx = hy.T
    
    Gx = convolve(img, hx, mode='nearest')
    Gy = convolve(img, hy, mode='nearest')
    
    gradientImg = np.hypot(Gx, Gy);
        
    return Gx, Gy, gradientImg


In [5]:
def getEdgePixels(gradientImg, edgeThresh): 
    Gmax = np.max(gradientImg[:])
    if not edgeThresh :
        # Python cv2 doen't take float image for otsu, 
        # Thus, we convert it into np.uint8 for thresholding for a while 
        edgeThresh, _ = cv2.threshold(np.asarray(gradientImg/Gmax*255, dtype=np.uint8),0,255, cv2.THRESH_OTSU)
        edgeThresh = edgeThresh/255

    t = Gmax * edgeThresh
    Ey, Ex = np.nonzero(gradientImg > t)

    return Ex, Ey

In [6]:
def bsxfun_1d(row, col, mode = 'times') : 
    
    mat_from_row = matlib.repmat(row, col.shape[0], 1)
    mat_from_col = matlib.repmat(col, row.shape[0], 1).T
    
    if mode == 'times':
        mat = np.multiply(mat_from_row, mat_from_col)
        
    elif mode == 'plus':
        mat = mat_from_row + mat_from_col
    
    return mat

In [7]:
def imreconstruct(marker: np.ndarray, mask: np.ndarray, radius: int = 1):
    """Iteratively expand the markers white keeping them limited by the mask during each iteration.

    :param marker: Grayscale image where initial seed is white on black background.
    :param mask: Grayscale mask where the valid area is white on black background.
    :param radius Can be increased to improve expansion speed while causing decreased isolation from nearby areas.
    :returns A copy of the last expansion.
    Written By Semnodime.
    """
    kernel = np.ones(shape=(radius * 2 + 1,) * 2, dtype=np.uint8)
    while True:
        expanded = cv2.dilate(src=marker, kernel=kernel)
        cv2.bitwise_and(src1=expanded, src2=mask, dst=expanded)

        # Termination criterion: Expansion didn't change the image at all
        if (marker == expanded).all():
            return expanded
        marker = expanded

In [8]:
def chaccum(A, radiusRange, method = 'phasecode',  objPolarity = 'bright', edgeThresh = [] ) : 
    
    maxNumElemNHoodMat = 1e6

    # all(A(:) == A(1));
    # Assume that image is not flat 

    # Get the input image in the correct format
    A = getGrayImage(A);

    ## Calculate gradient
    Gx, Gy, gradientImg = imgradient(A);

    Ex, Ey = getEdgePixels(gradientImg, edgeThresh);
    E = np.stack((Ey, Ex))

    # Not sure np.ravel_multi_index is equivalent with Matlab Function sub2ind
    idxE = np.ravel_multi_index(E, tuple(gradientImg.shape), mode='raise', order='C') 
    
    # Adding 0.0001 is to include end point of the radius
    radiusRange = np.arange(radiusRange[0], radiusRange[1]+0.0001, 0.5)

    if objPolarity == 'bright' :
        RR = radiusRange
    elif objPolarity == 'dark' :
        RR = - radiusRange

    ### This implementation only includes 'phasecode' mode 

    lnR = np.log(radiusRange);
    phi = ((lnR - lnR[0])/(lnR[-1] - lnR[0])*2*np.pi) - np.pi; # Modified form of Log-coding from Eqn. 8 in [3]

    Opca = np.exp(np.sqrt(-1+0j)*phi);
    w0 = Opca/(2*np.pi*radiusRange);

    xcStep = int(maxNumElemNHoodMat//RR.shape[0])

    lenE = Ex.shape[0]
    M, N = A.shape[0], A.shape[1]
    accumMatrix = np.zeros((M,N))
    
    for i in range(0, lenE+1, xcStep) : 

        Ex_chunk = Ex[i:int(np.min((i+xcStep-1,lenE)))]
        Ey_chunk = Ey[i:int(np.min((i+xcStep-1,lenE)))]
        idxE_chunk = idxE[i:int(np.min((i+xcStep-1,lenE)))]

        chunckX = np.divide(Gx[np.unravel_index(idxE_chunk, Gx.shape, 'C')],
                  gradientImg[np.unravel_index(idxE_chunk, gradientImg.shape, 'C')])

        chunckY = np.divide(Gy[np.unravel_index(idxE_chunk, Gy.shape, 'C')],
                  gradientImg[np.unravel_index(idxE_chunk, gradientImg.shape, 'C')])

        xc = matlib.repmat(Ex_chunk, RR.shape[0], 1).T + bsxfun_1d(-RR, chunckX) # Eqns. 10.3 & 10.4 from Machine Vision by E. R. Davies
        yc = matlib.repmat(Ey_chunk, RR.shape[0], 1).T + bsxfun_1d(-RR, chunckY)

        xc = np.round(xc)
        yc = np.round(yc)

        w = matlib.repmat(w0, xc.shape[0], 1)

        ## Determine which edge pixel votes are within the image domain
        # Record which candidate center positions are inside the image rectangle.
        M, N = A.shape[0], A.shape[1]

        inside = (xc >= 1) & (xc <= N) & (yc >= 1) & (yc < M)
        # Keep rows that have at least one candidate position inside the domain.
        rows_to_keep = np.any(inside, 1);
        xc = xc[rows_to_keep,:];
        yc = yc[rows_to_keep,:];
        w = w[rows_to_keep,:];
        inside = inside[rows_to_keep,:];

        ## Accumulate the votes in the parameter plane
        xc = xc[inside]
        yc = yc[inside]
        
        xyc = np.asarray(np.stack((xc[:], yc[:])), dtype= np.int64)

        accumMatrix = accumMatrix + accum(xyc.T, w[inside], size=[M, N])


#         del xc, yc, w # These are cleared to create memory space for the next loop. Otherwise out-of-memory at xc = bsxfun... in the next loop.
    
    return accumMatrix, gradientImg

In [9]:
def accum(accmap, a, size=None ): 
    
    if not size : 
        result = np.zeros((np.max(accmap[:, 0]), np.max(accmap[:, 1])), dtype=np.complex_) 
    
    else :
        result = np.zeros(size, dtype=np.complex_) 
        
    a = a[:, np.newaxis]
    concat = np.hstack((accmap, a))
    concat = concat[concat[:, 0].argsort()] # 이에 대해 sort
    
    df = pd.DataFrame(concat)
    x = df.groupby([0, 1]).sum().reset_index()[range(3)] #range(5) adjusts ordering 
    
    x_0 = np.asarray(x[0], dtype = np.int64) - 1
    x_1 = np.asarray(x[1], dtype = np.int64) - 1

    result[x_1, x_0] = x[2]    
    

    return result


In [10]:
def chcenters(accumMatrix, accumThresh) : 
    suppThreshold = accumThresh
    medFiltSize = 5
    kernel = np.ones((medFiltSize, medFiltSize),np.float32)/medFiltSize**2
    Hd = cv2.filter2D(accumMatrix,-1,kernel)

    suppThreshold = np.max(suppThreshold - np.spacing(suppThreshold), 0)

    Hd = imreconstruct(Hd-suppThreshold, Hd)

    lm = scipy.ndimage.filters.maximum_filter(Hd, size=3)
    bw = lm > np.max(lm)*2/3
    label_bw = label(bw)

    s = regionprops_table(label_bw, properties=['label','centroid'])
    centers = np.stack((s['centroid-0' ], s['centroid-1']))
    
    centers_int = np.asarray(centers, dtype = np.int64)
    metric = Hd[centers_int[0], centers_int[1]]
    
    # Sort the centers in descending order of metric    
    
    metric_sorted=np.sort(metric)[::-1]
    sortIndx=np.argsort(metric) 
    centers_sorted = np.fliplr(np.stack((np.take(centers[0,:], sortIndx), np.take(centers[1,:], sortIndx))))
    
    return centers_sorted , metric_sorted


In [11]:
def chradiiphcode(centers, accumMatrix, radiusRange) :
    centers_int = np.asarray(centers, dtype = np.int64)
    cenPhase = np.angle(accumMatrix[centers_int[0], centers_int[1]])
    lnR = np.log(radiusRange);
    r_estimated = np.exp(((cenPhase + np.pi)/(2*np.pi)*(lnR[-1] - lnR[0])) + lnR[0]) # Inverse of modified form of Log-coding from Eqn. 8 in [1]
    
    return r_estimated


In [12]:
def imfindcircles(A, radiusRange,  ObjectPolarity = 'bright', sensitivity = 0.95):

    [accumMatrix, gradientImg] = chaccum(A, radiusRange, objPolarity=ObjectPolarity);

    ## Estimate the centers
    accumThresh = 1 - sensitivity;

    accumMatrix = abs(accumMatrix);

    centers, metric = chcenters(accumMatrix, accumThresh)

    ## Assume Image is flat 
    ## Sigma is not specified 
    idx2Keep = np.nonzero(metric >= accumThresh)
    centers = centers[:, idx2Keep]
    metric = metric[idx2Keep]

    r_estimated = chradiiphcode(centers, accumMatrix, radiusRange)
    
    return centers, r_estimated, metric

In [13]:
start_p = time.time()

sensitivity = 0.96
centers = []
r_estimated = []
metric = []
radiusRange = [35, 45]

A = imread(img_list[0]) 

centers, r_estimated, metric = imfindcircles(A, radiusRange,  ObjectPolarity = 'bright', sensitivity = 0.95)
